In [1]:
from dotenv import load_dotenv

from pathlib import Path
import os 
import sys
import numpy as np
import json

load_dotenv()
GEMINI_KEY = os.getenv('GEMINI_KEY')

root = Path().resolve().parent

root_str = str(root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)

if Path.cwd() != root:
    os.chdir(root)



from google import genai

genai_client = genai.Client(
    api_key=GEMINI_KEY,
)

### Generation

In [3]:

response = genai_client.models.generate_content(
    model="gemma-3-12b-it",
    contents="Roses are red...",
)

response.candidates[0].content.parts[0].text

'Roses are red... Violets are blue... \n'

### Troubleshoot invalid json

In [2]:
import json

string = "```json\n{\"api_name\":\"RegisterUser\",\"parameters\":{\"username\":\"johnsmith123\",\"password\":\"1234password\",\"email\":\"johnsmith123@example.com\"}}\n```"

json.loads(string)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
with open("data/generated/1-rotowire/generated.json", "r") as f:
    data = json.load(f)

json.loads(data[0]["generated_output"].replace("```json", "").replace("```", ""))

{'teams': [{'team': 'Atlanta Hawks',
   'total_points': 107,
   'points_in_2nd_quarter': 31,
   'percentage_of_field_goals': 49,
   'number_of_team_assists': 29,
   'wins': 9,
   'losses': 2},
  {'team': 'Milwaukee Bucks',
   'total_points': 100,
   'points_in_2nd_quarter': 9,
   'wins': 5,
   'losses': 5}],
 'players': [{'player': 'Giannis Antetokounmpo',
   'points': 26,
   'total_rebounds': 15,
   'assists': 7},
  {'player': 'Jabari Parker',
   'points': 23,
   'total_rebounds': 8,
   'assists': 3,
   'steals': 3},
  {'player': 'Tony Snell', 'points': 11, 'total_rebounds': 4, 'assists': 3},
  {'player': 'Mirza Teletovic',
   'points': 14,
   'field_goals_made': 5,
   'field_goals_attempted': 10},
  {'player': 'Paul Millsap', 'points': 21, 'total_rebounds': 8, 'assists': 3},
  {'player': 'Mike Muscala', 'points': 16, 'total_rebounds': 5, 'assists': 3},
  {'player': 'Kyle Korver', 'points': 11, 'total_rebounds': 6, 'assists': 5}]}

In [5]:
from src.evaluate import assess_json_valid

s = """```json
{
    'article_title': "Bibs Ekkel",
    'associated_acts': "tziganka; balalaika potpourri; russian tornado",
    'background': "mixed polish-english",
    'birth_date': "1946-01-11",
    'birth_name': "",
    'genre': "russian folk music",
    'instrument': "balalaika",
    'name': "Bibs Ekkel",
    'occupation': "balalaika player; lecturer; music adviser",
    'origin': "brighton",
    'years_active': "19751946"
}
```"""

assess_json_valid(s)

Invalid JSON string: ```json
{
    'article_title': "Bibs Ekkel",
    'associated_acts': "tziganka; balalaika potpourri; russian tornado",
    'background': "mixed polish-english",
    'birth_date': "1946-01-11",
    'birth_name': "",
    'genre': "russian folk music",
    'instrument': "balalaika",
    'name': "Bibs Ekkel",
    'occupation': "balalaika player; lecturer; music adviser",
    'origin': "brighton",
    'years_active': "19751946"
}
``` - Fixing


(0,
 {'article_title': 'Bibs Ekkel',
  'associated_acts': 'tziganka; balalaika potpourri; russian tornado',
  'background': 'mixed polish-english',
  'birth_date': '1946-01-11',
  'birth_name': '',
  'genre': 'russian folk music',
  'instrument': 'balalaika',
  'name': 'Bibs Ekkel',
  'occupation': 'balalaika player; lecturer; music adviser',
  'origin': 'brighton',
  'years_active': '19751946'})

In [ ]:
import json

s = """```json
{
    'article_title': "Bibs Ekkel",
    'associated_acts': "tziganka; balalaika potpourri; russian tornado",
    'background': "mixed polish-english",
    'birth_date': "1946-01-11",
    'birth_name': "",
    'genre': "russian folk music",
    'instrument': "balalaika",
    'name': "Bibs Ekkel",
    'occupation': "balalaika player; lecturer; music adviser",
    'origin': "brighton",
    'years_active': "19751946"
}
```"""

a = s.replace("```json", "").replace("```", "")
json.loads(a)

JSONDecodeError: Expecting property name enclosed in double quotes: line 3 column 5 (char 7)

In [10]:
print(a)
json.loads(a)


{
    'article_title': "Bibs Ekkel",
    'associated_acts': "tziganka; balalaika potpourri; russian tornado",
    'background': "mixed polish-english",
    'birth_date': "1946-01-11",
    'birth_name': "",
    'genre': "russian folk music",
    'instrument': "balalaika",
    'name': "Bibs Ekkel",
    'occupation': "balalaika player; lecturer; music adviser",
    'origin': "brighton",
    'years_active': "19751946"
}



JSONDecodeError: Expecting property name enclosed in double quotes: line 3 column 5 (char 7)

# Results

In [14]:
import pandas as pd
import json
from pprint import pprint

results = json.load(open("results/bench_results.json"))

reshuffle = {}
for metric in ["is_valid", "correctness", "compliance"]:
    reshuffle[metric] = {
        model: {
            task: results[model][task][metric]
            for task in results[model]
        }
        for model in results
    }

pd.DataFrame(reshuffle["is_valid"])

,gemini-2.5-flash-preview-05-20,gemma-3-12b-it,gemma-3-4b-it,gemma-3-27b-it
1-rotowire,0.800000,0.800,0.800,0.800
2-wiki_bio,0.986000,0.546,0.732,0.588
3-few_nerd,1.000000,0.800,0.984,0.800
4-TOPv1,0.953333,0.795,0.800,0.800
5-api_bank,0.820000,0.979,0.405,0.640
6-reasoning/GSM8K,0.794000,0.828,0.704,0.890
6-reasoning/last_letter,0.800000,0.800,0.800,0.800


In [16]:
pd.DataFrame(reshuffle["compliance"])


,gemini-2.5-flash-preview-05-20,gemma-3-12b-it,gemma-3-4b-it,gemma-3-27b-it
1-rotowire,1.000000,0.975,0.947,0.990
2-wiki_bio,0.964470,0.970,0.967,0.973
3-few_nerd,1.000000,0.995,1.000,1.000
4-TOPv1,0.933333,0.760,0.767,0.900
5-api_bank,0.995000,0.996,0.965,0.989
6-reasoning/GSM8K,1.000000,1.000,0.975,1.000
6-reasoning/last_letter,1.000000,1.000,1.000,1.000


In [18]:
pd.DataFrame(reshuffle["correctness"])


,gemini-2.5-flash-preview-05-20,gemma-3-12b-it,gemma-3-4b-it,gemma-3-27b-it
1-rotowire,0.565141,0.544,0.505,0.504
2-wiki_bio,0.326844,0.324,0.348,0.333
3-few_nerd,0.599250,0.394,0.425,0.384
4-TOPv1,0.639820,0.510,0.491,0.602
5-api_bank,0.907213,0.834,0.797,0.799
6-reasoning/GSM8K,0.930000,0.890,0.840,0.910
6-reasoning/last_letter,0.980000,0.810,0.170,0.840
